In [1090]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [1091]:
users= pd.read_csv('E:/Учёба/DA/EduTech/Data_srt1/users.csv', delimiter=';') #импорт таблицы users (студенты), прописываем адрес хранения файла
users.head(5)

,unti_id,userID,course_id,flow_num,tgBot,timeZone,age,M1_progress,M1_attestation,M1_attestation_date,M2_progress,M2_attestation,course_progress2,M2_attestation_date,course_attestation,course_attestation_date
0,1312039.0,71,82,1.0,подключен,+03:00,NaN,19,Не сдана,NaN,0,Не сдана,13,NaN,Не сдана,NaN
1,1309687.0,171,82,1.0,подключен,+03:00,NaN,94,100,09.07.2024 14:12,71,100,88,28.08.2024 13:33,100,28.08.2024 13:33
2,1046971.0,192,82,1.0,подключен,+03:00,NaN,100,100,14.08.2024 18:26,71,100,92,06.09.2024 15:16,100,12.09.2024 12:17
3,470195.0,193,76,1.0,подключен,+03:00,44.0,Нет данных,Не сдана,NaN,Нет данных,Не сдана,70,NaN,Не сдана,NaN
4,1825188.0,1165,82,1.0,подключен,+03:00,NaN,81,100,12.07.2024 0:00,0,Не сдана,57,NaN,100,28.08.2024 13:34


In [1092]:
users.info() #запоминаем количество строк, чтобы в дальнейшем контролировать количество студентов

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 608 entries, 0 to 607
Data columns (total 16 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   unti_id                  605 non-null    float64
 1   userID                   608 non-null    int64  
 2   course_id                608 non-null    int64  
 3   flow_num                 608 non-null    float64
 4   tgBot                    608 non-null    object 
 5   timeZone                 608 non-null    object 
 6   age                      10 non-null     float64
 7   M1_progress              608 non-null    object 
 8   M1_attestation           608 non-null    object 
 9   M1_attestation_date      253 non-null    object 
 10  M2_progress              608 non-null    object 
 11  M2_attestation           608 non-null    object 
 12  course_progress2         608 non-null    int64  
 13  M2_attestation_date      206 non-null    object 
 14  course_attestation       6

In [1093]:
#приводим столбец "дата аттестации по курсу" к формату datetime
users['course_attestation_date'] = pd.to_datetime(users['course_attestation_date'])

C:\Users\Инга\AppData\Local\Temp\ipykernel_2908\4034825759.py:2: UserWarning: Parsing dates in %d.%m.%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  users['course_attestation_date'] = pd.to_datetime(users['course_attestation_date'])


In [1094]:
users.head(2)

,unti_id,userID,course_id,flow_num,tgBot,timeZone,age,M1_progress,M1_attestation,M1_attestation_date,M2_progress,M2_attestation,course_progress2,M2_attestation_date,course_attestation,course_attestation_date
0,1312039.0,71,82,1.0,подключен,+03:00,NaN,19,Не сдана,NaN,0,Не сдана,13,NaN,Не сдана,NaT
1,1309687.0,171,82,1.0,подключен,+03:00,NaN,94,100,09.07.2024 14:12,71,100,88,28.08.2024 13:33,100,2024-08-28 13:33:00


In [1095]:
users.isna().sum()

unti_id                      3
userID                       0
course_id                    0
flow_num                     0
tgBot                        0
timeZone                     0
age                        598
M1_progress                  0
M1_attestation               0
M1_attestation_date        355
M2_progress                  0
M2_attestation               0
course_progress2             0
M2_attestation_date        402
course_attestation           0
course_attestation_date    256
dtype: int64

In [1096]:
# переименовываем колонки в соотвествии с названиями в нашей базе
users.rename(columns = {'userID': 'user_id', 'course_progress2': 'course_progress'}, inplace = True)

In [1097]:
# берем только данные об итоговой аттестации, можно при необходимости добавить данные по отдельным модулям
users = users[['user_id', 'course_id', 'course_progress', 'course_attestation', 'course_attestation_date']].copy()

In [1098]:
#для ускорения расчётов или при необходимости можно выбрать конкретного студента по user_id
#users = users.query('user_id == 5596') 
#users = users.reset_index(drop=True)
#users.head(2)

In [1099]:
authorization= pd.read_csv('E:/Учёба/DA/EduTech/Data_srt1/authorization.csv')  # импорт талицы authorization (логи авторизаций), прописываем адрес хранения файла
authorization.head(5)

,user_id,created_at,user_agent,window_size
0,193,2023-05-06 21:15:12,Mozilla/5.0 (Linux; U; Android 13; ru-ru; Xiao...,1138x531
1,193,2023-05-06 21:39:44,Mozilla/5.0 (Linux; U; Android 13; ru-ru; Xiao...,1138x531
2,193,2023-05-08 11:20:51,Mozilla/5.0 (Linux; arm_64; Android 13; 210511...,1138x557
3,193,2023-05-22 22:56:50,Mozilla/5.0 (Linux; arm_64; Android 13; 210511...,1138x557
4,193,2023-05-22 22:56:52,Mozilla/5.0 (Linux; arm_64; Android 13; 210511...,1138x557


In [1100]:
authorization.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94702 entries, 0 to 94701
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      94702 non-null  int64 
 1   created_at   94702 non-null  object
 2   user_agent   94702 non-null  object
 3   window_size  94702 non-null  object
dtypes: int64(1), object(3)
memory usage: 2.9+ MB


In [1101]:
authorization.drop(['user_agent', 'window_size'], axis=1, inplace=True)

In [1102]:
#рассчитывем количество авторизаций для каждого студента
authorization2 = authorization.groupby(['user_id']).agg('count')
authorization2.rename(columns = {'created_at': 'num_of_auth'}, inplace = True)
authorization2.head(5)

,num_of_auth
user_id,
71,33
171,121
192,478
193,3616
1165,199


In [1103]:
us_auth = pd.merge(users, authorization, on='user_id', how='left') # объединяем таблицы users и authorization

In [1104]:
us_auth.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94712 entries, 0 to 94711
Data columns (total 6 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   user_id                  94712 non-null  int64         
 1   course_id                94712 non-null  int64         
 2   course_progress          94712 non-null  int64         
 3   course_attestation       94712 non-null  object        
 4   course_attestation_date  71479 non-null  datetime64[ns]
 5   created_at               94702 non-null  object        
dtypes: datetime64[ns](1), int64(3), object(2)
memory usage: 4.3+ MB


In [1105]:
us_auth.rename(columns = {'created_at': 'first_authirization'}, inplace = True)

In [1106]:
us_auth.head(2)

,user_id,course_id,course_progress,course_attestation,course_attestation_date,first_authirization
0,71,82,13,Не сдана,NaT,2024-05-27 14:03:55
1,71,82,13,Не сдана,NaT,2024-05-27 14:07:14


In [1107]:
us_auth.drop_duplicates(subset='user_id', keep='first', inplace=True) # оставляем дату первой авторизации в качестве даты регистрации

In [1108]:
df_merged = pd.merge(us_auth, authorization2, on='user_id', how='left') # добавляем количество авторизаций

In [1109]:
df_merged.head(2)

,user_id,course_id,course_progress,course_attestation,course_attestation_date,first_authirization,num_of_auth
0,71,82,13,Не сдана,NaT,2024-05-27 14:03:55,33.0
1,171,82,88,100,2024-08-28 13:33:00,2024-06-05 15:21:40,121.0


In [1110]:
df_merged.info()  # сверяем количество строк с таблицей users

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 608 entries, 0 to 607
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   user_id                  608 non-null    int64         
 1   course_id                608 non-null    int64         
 2   course_progress          608 non-null    int64         
 3   course_attestation       608 non-null    object        
 4   course_attestation_date  352 non-null    datetime64[ns]
 5   first_authirization      598 non-null    object        
 6   num_of_auth              598 non-null    float64       
dtypes: datetime64[ns](1), float64(1), int64(3), object(2)
memory usage: 33.4+ KB


In [1111]:
webinars_logs= pd.read_csv('E:/Учёба/DA/EduTech/Data_srt1/webinars_logs.csv') # импорт таблицы webinars_logs (логи вебинаров), прописываем адрес хранения файла
webinars_logs.head(5)

,userId,dateTime,eventName,webinarId,формат подключения,вводный вебинар,module
0,2971,2024-06-11 16:02:57,Подключение,16806,офлайн,0,1
1,2971,2024-06-11 16:43:03,Отключение,16806,офлайн,0,1
2,2971,2024-06-12 10:02:54,Подключение,16806,офлайн,0,1
3,2971,2024-06-12 10:24:16,Отключение,16806,офлайн,0,1
4,2971,2024-06-12 16:43:30,Подключение,16807,офлайн,0,1


In [1112]:
# удаляем ненужный столбец, переименовываем колонки в соотвествии с названиями в нашей базе
webinars_logs.drop(['module'], axis=1, inplace=True)
webinars_logs.rename(columns = {'dateTime': 'datetime', 'userId': 'user_id', 'eventName': 'event_name', 'webinarId': 'webinar_id', 'вводный вебинар': 'webinar_int'}, inplace = True)

In [1113]:
webinars_logs['datetime'] = pd.to_datetime(webinars_logs['datetime']) # приводим к формату datetime

In [1114]:
webinars_logs.head(5)

,user_id,datetime,event_name,webinar_id,формат подключения,webinar_int
0,2971,2024-06-11 16:02:57,Подключение,16806,офлайн,0
1,2971,2024-06-11 16:43:03,Отключение,16806,офлайн,0
2,2971,2024-06-12 10:02:54,Подключение,16806,офлайн,0
3,2971,2024-06-12 10:24:16,Отключение,16806,офлайн,0
4,2971,2024-06-12 16:43:30,Подключение,16807,офлайн,0


In [1115]:
webinar_vvod = webinars_logs[['user_id','webinar_int']].copy() # копируем колонку с информацией о просмотрах вводного вебинара

In [1116]:
webinar_vvod.drop_duplicates(subset='user_id', keep='first', inplace=True) # удаляем дубликаты
webinar_vvod.head(20)

,user_id,webinar_int
0,2971,0
16,3001,0
18,3014,0
330,3133,0
494,3237,0
580,3304,0
712,3334,0
724,3564,0
726,3597,0
732,3607,0


Расчёт продолжительности просмотра вебинаров

In [1117]:
# выделяем подключения
connected = webinars_logs[webinars_logs['event_name'] == 'Подключение'].reset_index(drop=True) # выбираем время подключения
connected['date'] = connected['datetime'].apply(lambda x: x.date())
con = connected[['user_id', 'date', 'datetime', 'webinar_id']].copy()
con.sort_values(['user_id', 'webinar_id', 'datetime'], inplace=True)
con['session_id'] = con.groupby(['user_id', 'webinar_id']).cumcount()  

# выделяем отключения
disconnected = webinars_logs[webinars_logs['event_name'] == 'Отключение'].reset_index(drop=True) # выбираем время отключения
disconnected['date'] = disconnected['datetime'].apply(lambda x: x.date())
dis = disconnected[['user_id', 'date', 'datetime','webinar_id']].copy()
dis.sort_values(['user_id', 'webinar_id', 'datetime'], inplace=True)
dis['session_id'] = dis.groupby(['user_id', 'webinar_id']).cumcount()  

# объединяем подключения и отключения
webinar_viewing = pd.merge(con, dis, on=['user_id', 'webinar_id', 'date', 'session_id'], suffixes=('_connect', '_disconnect'))

# Удаляем временные промежуточные идентификаторы, если они больше не нужны
webinar_viewing.drop(columns=['session_id'], inplace=True)
webinar_viewing.head(15)

,user_id,date,datetime_connect,webinar_id,datetime_disconnect
0,71,2024-06-04,2024-06-04 20:03:41,16746,2024-06-04 20:23:36
1,71,2024-06-05,2024-06-05 14:10:23,16746,2024-06-05 14:10:25
2,71,2024-06-05,2024-06-05 15:06:07,16746,2024-06-05 15:07:24
3,71,2024-06-07,2024-06-07 08:23:49,16747,2024-06-07 08:23:55
4,71,2024-06-07,2024-06-07 08:24:02,16747,2024-06-07 14:37:40
5,71,2024-06-05,2024-06-05 14:55:19,16910,2024-06-05 14:56:30
6,171,2024-09-30,2024-09-30 14:33:25,15996,2024-09-30 14:35:48
7,171,2024-06-06,2024-06-06 20:01:17,16747,2024-06-06 20:06:52
8,171,2024-06-06,2024-06-06 20:06:57,16747,2024-06-06 20:08:26
9,171,2024-06-08,2024-06-08 12:53:17,16747,2024-06-08 13:05:54


суммарная длительность просмотров вебинаров по юзерам

In [1118]:
# избегаем отрицательных значений в случае отсутствия логов
webinar_viewing['w_view_hours'] = np.where(
    webinar_viewing['datetime_disconnect'] > webinar_viewing['datetime_connect'],
    (webinar_viewing['datetime_disconnect'] - webinar_viewing['datetime_connect']).dt.total_seconds() / 3600,
    0  # или np.nan если вы хотите оставить пустым для значений, которые не соответствуют условию
)

webinar_viewing.head(10)

,user_id,date,datetime_connect,webinar_id,datetime_disconnect,w_view_hours
0,71,2024-06-04,2024-06-04 20:03:41,16746,2024-06-04 20:23:36,0.331944
1,71,2024-06-05,2024-06-05 14:10:23,16746,2024-06-05 14:10:25,0.000556
2,71,2024-06-05,2024-06-05 15:06:07,16746,2024-06-05 15:07:24,0.021389
3,71,2024-06-07,2024-06-07 08:23:49,16747,2024-06-07 08:23:55,0.001667
4,71,2024-06-07,2024-06-07 08:24:02,16747,2024-06-07 14:37:40,6.227222
5,71,2024-06-05,2024-06-05 14:55:19,16910,2024-06-05 14:56:30,0.019722
6,171,2024-09-30,2024-09-30 14:33:25,15996,2024-09-30 14:35:48,0.039722
7,171,2024-06-06,2024-06-06 20:01:17,16747,2024-06-06 20:06:52,0.093056
8,171,2024-06-06,2024-06-06 20:06:57,16747,2024-06-06 20:08:26,0.024722
9,171,2024-06-08,2024-06-08 12:53:17,16747,2024-06-08 13:05:54,0.210278


In [1119]:
# считаем суммарное время просмотра вебинаров для одного студента
webinar_viewing_1 = webinar_viewing.groupby(['user_id'])[['w_view_hours']].agg('sum')
webinar_viewing_1.head(5)

,w_view_hours
user_id,
71,6.602500
171,14.450556
192,26.158333
193,353.388889
1165,1.425556


время просмотра отдельных вебинаров (далее не используется, но может пригодиться для анализа в будущем)

In [1120]:
webinar_viewing['w_view_hours'] = (webinar_viewing['datetime_disconnect'] - webinar_viewing['datetime_connect']).dt.total_seconds() /3600 # вычисляем продолжиельность просмотра, переводим в часы
webinar_viewing_2 = webinar_viewing.groupby(['user_id', 'webinar_id'])[['w_view_hours']].agg('sum')
webinar_viewing_2.head(10)

w_view_hours
user_id webinar_id              
71      16746           0.353889
        16747           6.228889
        16910           0.019722
171     15996           0.039722
        16747           0.328056
        16748           0.095556
        16749           0.147778
        16750           2.126389
        16751           0.293333
        16790           0.006667

In [1121]:
# Присоединяем просмотры вводных вебинаров
webinar_viewing_sum = pd.merge(df_merged, webinar_vvod, on='user_id', how='left')
webinar_viewing_sum.head(2)

,user_id,course_id,course_progress,course_attestation,course_attestation_date,first_authirization,num_of_auth,webinar_int
0,71,82,13,Не сдана,NaT,2024-05-27 14:03:55,33.0,0.0
1,171,82,88,100,2024-08-28 13:33:00,2024-06-05 15:21:40,121.0,1.0


In [1122]:
# Присоединяем продолжительность просмотра вебинаров
webinar_viewing_sum = pd.merge(webinar_viewing_sum, webinar_viewing_1, on='user_id', how='left')
webinar_viewing_sum.head(2)

,user_id,course_id,course_progress,course_attestation,course_attestation_date,first_authirization,num_of_auth,webinar_int,w_view_hours
0,71,82,13,Не сдана,NaT,2024-05-27 14:03:55,33.0,0.0,6.602500
1,171,82,88,100,2024-08-28 13:33:00,2024-06-05 15:21:40,121.0,1.0,14.450556


In [1123]:
webinar_viewing_sum.info() # проверяем, что количество строк не изменилось

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 608 entries, 0 to 607
Data columns (total 9 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   user_id                  608 non-null    int64         
 1   course_id                608 non-null    int64         
 2   course_progress          608 non-null    int64         
 3   course_attestation       608 non-null    object        
 4   course_attestation_date  352 non-null    datetime64[ns]
 5   first_authirization      598 non-null    object        
 6   num_of_auth              598 non-null    float64       
 7   webinar_int              568 non-null    float64       
 8   w_view_hours             567 non-null    float64       
dtypes: datetime64[ns](1), float64(3), int64(3), object(2)
memory usage: 42.9+ KB


In [1124]:
activity_history_viewed = pd.read_csv('E:/Учёба/DA/EduTech/Data_srt1/activity_history_viewed.csv') # импорт таблицы activity_history_viewed (история просмотров), прописываем адрес хранения файла
activity_history_viewed.tail(2)

,user_id,created_at,page_type,page_id,module,attestation,activity_type
648991,5788,2024-07-08 07:20:17,активность,17315,1,0,slide
648992,5788,2024-07-07 22:37:03,активность,17333,1,0,video


In [1125]:
# копируем нужные столбцы с идентификатором пользователя, датой и временем просмотра, типом страницы
activity_sum = activity_history_viewed[['user_id', 'created_at','page_type']].copy()

In [1126]:
# приводим тип данных в столбце "дата просмотра" к формату datetime
activity_sum['created_at'] = pd.to_datetime(activity_sum['created_at'])

In [1127]:
# смотрим количество активностей в таблице
activity_sum['user_id'].count()

648993

In [1128]:
# считаем суммарное количество просмотров страниц с типом "занятие" для каждого пользователя
sum_task_id = activity_sum.loc[(activity_sum['page_type'] == 'занятие')]
sum_task_id = sum_task_id.groupby(['user_id', 'page_type']).agg('count')
sum_task_id.rename(columns = {'created_at': 'view_task_id'}, inplace = True)
sum_task_id.head(2)

,,view_task_id
user_id,page_type,
71,занятие,119
171,занятие,251


In [1129]:
# считаем суммарное количество просмотров страниц с типом "активность" для каждого пользователя
sum_activity_id = activity_sum.loc[(activity_sum['page_type'] == 'активность')]
sum_activity_id = sum_activity_id.groupby(['user_id', 'page_type']).agg('count')
sum_activity_id.rename(columns = {'created_at': 'view_activity_id'}, inplace = True)
sum_activity_id.head(2)

,,view_activity_id
user_id,page_type,
71,активность,90
171,активность,496


In [1130]:
# объединяем предыдущие две таблицы
activity_sum = pd.merge(sum_task_id, sum_activity_id, on=['user_id'], how='left')
activity_sum.head(2)

,view_task_id,view_activity_id
user_id,,
71,119,90.0
171,251,496.0


In [1131]:
#объединяем сводную таблицу с количеством просмотренных страниц
df_merged = pd.merge(webinar_viewing_sum, activity_sum, on=['user_id'], how='left')
df_merged.head(5)

,user_id,course_id,course_progress,course_attestation,course_attestation_date,first_authirization,num_of_auth,webinar_int,w_view_hours,view_task_id,view_activity_id
0,71,82,13,Не сдана,NaT,2024-05-27 14:03:55,33.0,0.0,6.602500,119.0,90.0
1,171,82,88,100,2024-08-28 13:33:00,2024-06-05 15:21:40,121.0,1.0,14.450556,251.0,496.0
2,192,82,92,100,2024-09-12 12:17:00,2024-06-02 02:21:58,478.0,0.0,26.158333,636.0,1136.0
3,193,76,70,Не сдана,NaT,2023-05-06 21:15:12,3616.0,0.0,353.388889,1725.0,3247.0
4,1165,82,57,100,2024-08-28 13:34:00,2024-05-18 12:20:01,199.0,0.0,1.425556,283.0,513.0


In [1132]:
# собран датасет с информацией для каждого студента о прогрессе по курсу, сдаче аттестации и дате аттестации, дате первой авторизации, количестве авторизаций
# просмотре вводного вебинара, количестве часов просмотра вебинаров, количестве просмотренных заданий и активностей
#делаем копию датасета, используем её для сборки сводного датасета в самом конце
for_overwiew = df_merged.copy()

Обрабатываем результаты выполнения заданий и расписание

In [1133]:
schedule = pd.read_csv('E:/Учёба/DA/EduTech/Data_srt1/schedule.csv')  # импорт таблицы schedule (расписание), прописываем адрес хранения файла

In [1134]:
# переименовываем столбцы согласно нашим названиям столбцов в базе
schedule.rename(columns={'taskID': 'task_id', 'activivtyID':'activity_id', 'dateShown': 'date_shown', 'isAttestation': 'is_attestation'}, inplace=True)

In [1135]:
# приводим формат данных в стобце "дата открытия задания" к datetime
schedule['date_shown'] = pd.to_datetime(schedule['date_shown'])

In [1136]:
# удаляем пустые значения
schedule.dropna(inplace = True)

In [1137]:
schedule.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1648 entries, 167 to 2172
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   course_id       1648 non-null   int64         
 1   type            1648 non-null   object        
 2   task_id         1648 non-null   int64         
 3   activity_id     1648 non-null   float64       
 4   activityType    1648 non-null   object        
 5   is_attestation  1648 non-null   int64         
 6   visibility      1648 non-null   object        
 7   flows           1648 non-null   int64         
 8   date_shown      1648 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(4), object(3)
memory usage: 128.8+ KB


In [1138]:
exercise_results = pd.read_csv('E:/Учёба/DA/EduTech/Data_srt1/exercise_results.csv') # импорт таблицы exercise_results (результаты выполнения заданий), прописываем свой адрес хранения файла
exercise_results.head(2)

,module,activityID,userId,createdAt,result,success
0,1,15590,2971,2024-06-12 16:54:12,100,1.0
1,1,17155,2971,2024-06-17 15:36:57,0,0.0


In [1139]:
# переименовываем столбцы согласно нашим названиям столбцов в базе
exercise_results.rename(columns={'userId':'user_id', 'activityID':'activity_id', 'taskID': 'task_id', 'createdAt': 'created_at'}, inplace=True)

In [1140]:
# приводим формат данных в стобце "дата выполнения задания" к datetime
exercise_results['created_at'] = pd.to_datetime(exercise_results['created_at'])

In [1141]:
#объединяем данные о выполнении заданий с номерами пользователей и курсов
df_merged_er = pd.merge(users[['user_id', 'course_id']], exercise_results, on='user_id', how='left')
df_merged_er.head(5)

,user_id,course_id,module,activity_id,created_at,result,success
0,71,82,1.0,15614.0,2024-06-05 15:38:39,100,1.0
1,71,82,1.0,15612.0,2024-06-05 15:39:38,100,1.0
2,171,82,1.0,15614.0,2024-06-06 12:52:44,100,1.0
3,171,82,1.0,15627.0,2024-06-13 23:04:50,100,1.0
4,171,82,1.0,15672.0,2024-06-24 11:55:30,0,0.0


In [1142]:
# присоединяем расписание
df_merged_sch = pd.merge(df_merged_er, schedule, on=['activity_id', 'course_id'], how='left')
df_merged_sch.head(5)

,user_id,course_id,module,activity_id,created_at,result,success,type,task_id,activityType,is_attestation,visibility,flows,date_shown
0,71,82,1.0,15614.0,2024-06-05 15:38:39,100,1.0,активность,1691.0,exercise,0.0,"""""",1.0,2024-06-04 20:00:00
1,71,82,1.0,15612.0,2024-06-05 15:39:38,100,1.0,активность,1691.0,questions,0.0,{},1.0,2024-06-04 20:00:00
2,171,82,1.0,15614.0,2024-06-06 12:52:44,100,1.0,активность,1691.0,exercise,0.0,"""""",1.0,2024-06-04 20:00:00
3,171,82,1.0,15627.0,2024-06-13 23:04:50,100,1.0,активность,1693.0,exercise,0.0,"""""",1.0,2024-06-11 20:00:00
4,171,82,1.0,15672.0,2024-06-24 11:55:30,0,0.0,активность,1695.0,exercise,0.0,"""""",1.0,2024-06-18 20:00:00


In [1143]:
# проверяем, что количество студентов не изменилось
df_merged_sch['user_id'].value_counts()

user_id
4905    127
4356    126
3517    123
1540    120
3568    120
       ... 
4744      1
4738      1
4698      1
5590      1
6443      1
Name: count, Length: 608, dtype: int64

In [1146]:
df_merged_sch.drop(['module', 'type', 'visibility', 'flows', 'is_attestation'], axis=1, inplace=True)

In [1147]:
df_merged_sch.head(2)

,user_id,course_id,activity_id,created_at,result,success,task_id,activityType,date_shown
0,71,82,15614.0,2024-06-05 15:38:39,100,1.0,1691.0,exercise,2024-06-04 20:00:00
1,71,82,15612.0,2024-06-05 15:39:38,100,1.0,1691.0,questions,2024-06-04 20:00:00


In [1148]:
# импорт таблицы activities_guide (справочник активностей), прописываем свой адрес хранения файла
activities_guide = pd.read_csv('E:/Учёба/DA/EduTech/Data_srt1/activities_catalog.csv')
activities_guide.head(2)

,courseID,Курс,Провайдер,Модуль,themeID,Тема,taskID,Занятие,taskPosition,Признак Аттестации,activityID,Тип Активности,Активность,Признак Обязательного,Видимость
0,76,Архитектор данных,1Т,1,332,Введение в архитектуру больших данных и операц...,1608,Введение в профессию. Основные принципы работы...,2,0,15569,slide,Задачи архитектора данных,0.0,{}
1,76,Архитектор данных,1Т,1,332,Введение в архитектуру больших данных и операц...,1608,Введение в профессию. Основные принципы работы...,2,0,15570,slide,Кто такие архитекторы данных?,0.0,{}


In [1149]:
# переименовываем названия столбцов в соотвествии с названиями в нашей базе данных
activities_guide.rename(columns={'courseId':'course_id', 'activityID':'activity_id', 'taskID': 'task_id', 'Признак Обязательного': 'obyaz_priznak', 'Признак Аттестации': 'is_attestation'}, inplace=True)

In [1150]:
# проверяем на нулевые значения
activities_guide.isnull().sum()

courseID             0
Курс                 0
Провайдер            0
Модуль               0
themeID              0
Тема                 0
task_id              0
Занятие              0
taskPosition         0
is_attestation       0
activity_id          0
Тип Активности       0
Активность          81
obyaz_priznak     1024
Видимость          342
dtype: int64

In [1151]:
# удаляем из справочника активностей строки с нулевыми значениями в колонке "обязательный признак"
activities_guide.dropna(axis=0, subset=['obyaz_priznak'], inplace=True)

In [1152]:
# проверяем, какие уникальные значения есть с колонке с номером курса
unique_vals = activities_guide['courseID'].unique()
print(unique_vals)

[76 77 82 83]


In [1153]:
# определяем количество обязательных заданий (в примере для курса 83 "Арзитектор данных")
# используем полученное значение для вычисления прогресса по формуле в самом конце
activities_guide.loc[(activities_guide['courseID'] == 83) &(activities_guide['is_attestation'] == 0), 'obyaz_priznak'].sum()

24.0

In [1154]:
# объединяем нужные нам столбцы справочника активностей со сводным дата-сетом
df_merged = pd.merge(activities_guide[['activity_id', 'obyaz_priznak', 'is_attestation']], df_merged_sch, on=['activity_id'], how='left')
df_merged.head(5)

,activity_id,obyaz_priznak,is_attestation,user_id,course_id,created_at,result,success,task_id,activityType,date_shown
0,15569,0.0,0,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaT
1,15570,0.0,0,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaT
2,15571,0.0,0,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaT
3,15572,0.0,0,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaT
4,15573,0.0,0,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaT


In [1155]:
# определяем тип данных столбца result
print(df_merged['result'].dtypes, "\n")

object 



In [1156]:
# проверяем, какие уникальные значения есть с колонке "результат"
unique_vals = df_merged['result'].unique()
print(unique_vals)

[nan '100' '57' '86' '71' '0' '67' '83' '80' '60' '90' '70' '78' '89' '75'
 '93' '79' '64' '95' '85' '88' '63' '50' '81' '94' '69' '38' '40'
 'Пропуск' '92' '87' '73' '96' '91' '74' '61' '82' '44' '20' '58']


In [1157]:
# заменяем словесные статусы на числовые, отличные от остальных
df_merged['result'] = df_merged['result'].apply(lambda x: -1 if x == 'Пропуск' else x)
#df_merged['result'] = df_merged['result'].apply(lambda x: -1 if x == 'На проверке' else x)

In [1158]:
# заполняем пропуски
df_merged = df_merged.fillna(0)

In [1159]:
# меняем тип данных столбца, чтобы производить с ним подсчёты
df_merged['result'].astype(str).astype(int)
df_merged['result'] = pd.to_numeric(df_merged['result'])
print(df_merged['result'].dtypes, "\n")

int64 



In [1160]:
df_merged.head(2)

,activity_id,obyaz_priznak,is_attestation,user_id,course_id,created_at,result,success,task_id,activityType,date_shown
0,15569,0.0,0,0.0,0.0,0,0,0.0,0.0,0,0
1,15570,0.0,0,0.0,0.0,0,0,0.0,0.0,0,0


In [1161]:
# считаем средний результат и успех выполнения каждой активности
sum_result = df_merged.groupby(['user_id', 'activity_id'])[['result', 'success']].agg('mean')
sum_result.head(2)

result  success
user_id activity_id                 
0.0     15479           0.0      0.0
        15480           0.0      0.0

In [1162]:
# переименовываем колонки
sum_result.rename(columns = {'result': 'mean_result', 'success': 'mean_success'}, inplace = True)

In [1163]:
# объединяем сводный датасет с данными о средних результатах
df_merged = pd.merge(df_merged, sum_result, on=['user_id', 'activity_id'], how='left')
df_merged.head(5)

,activity_id,obyaz_priznak,is_attestation,user_id,course_id,created_at,result,success,task_id,activityType,date_shown,mean_result,mean_success
0,15569,0.0,0,0.0,0.0,0,0,0.0,0.0,0,0,0.0,0.0
1,15570,0.0,0,0.0,0.0,0,0,0.0,0.0,0,0,0.0,0.0
2,15571,0.0,0,0.0,0.0,0,0,0.0,0.0,0,0,0.0,0.0
3,15572,0.0,0,0.0,0.0,0,0,0.0,0.0,0,0,0.0,0.0
4,15573,0.0,0,0.0,0.0,0,0,0.0,0.0,0,0,0.0,0.0


In [1164]:
# приводим тип данных колонки 'дата выполнения' к datetime
df_merged['created_at'] = pd.to_datetime(df_merged['created_at'], format="%Y-%m-%d %H:%M:%S.%f", errors='coerce' )

In [1165]:
# приводим тип данных колонки 'дата открытия' к datetime
df_merged['date_shown'] = pd.to_datetime(df_merged['date_shown'], format="%Y-%m-%d %H:%M:%S.%f", errors='coerce' )

In [1166]:
# считаем, сколько дней студенту потребовалось на выполнение активности
df_merged['delay_days'] = (df_merged['created_at'] - df_merged['date_shown']).dt.days
df_merged.head(5)

,activity_id,obyaz_priznak,is_attestation,user_id,course_id,created_at,result,success,task_id,activityType,date_shown,mean_result,mean_success,delay_days
0,15569,0.0,0,0.0,0.0,NaT,0,0.0,0.0,0,NaT,0.0,0.0,NaN
1,15570,0.0,0,0.0,0.0,NaT,0,0.0,0.0,0,NaT,0.0,0.0,NaN
2,15571,0.0,0,0.0,0.0,NaT,0,0.0,0.0,0,NaT,0.0,0.0,NaN
3,15572,0.0,0,0.0,0.0,NaT,0,0.0,0.0,0,NaT,0.0,0.0,NaN
4,15573,0.0,0,0.0,0.0,NaT,0,0.0,0.0,0,NaT,0.0,0.0,NaN


In [1167]:
# для тех случаев, когда активность выполнена раньше, чем открыта (когда активность открыта в начале модуля, а не согласно расписанию)
df_merged.loc[df_merged['delay_days'] < 0, 'delay_days'] = 1

In [1168]:
shed_res_act_delta_days = df_merged[(df_merged.is_attestation == 0)&(df_merged.result != -1)]
delta_days_mean = shed_res_act_delta_days.groupby(['user_id'])[['delay_days']].agg('mean')
delta_days_mean.rename(columns = {'delay_days': 'delay_days_mean'}, inplace = True)
delta_days_mean.head(2)

,delay_days_mean
user_id,
0.0,NaN
71.0,0.0


средний результат выполнения неаттестационных активностей

In [1169]:
shed_res_act_result = df_merged[(df_merged.is_attestation == 0)&(df_merged.result != -1)]
result_mean = shed_res_act_result.groupby(['user_id'])[['result']].agg('mean')
result_mean.rename(columns = {'result': 'result_mean'}, inplace = True)
result_mean.head(2)

,result_mean
user_id,
0.0,0.0
71.0,100.0


средний успех выполнения неаттестационных активностей

In [1170]:
shed_res_act_success = df_merged[(df_merged.is_attestation == 0)&(df_merged.result != -1)]
success_mean = shed_res_act_success.groupby(['user_id'])[['success']].agg('mean')
success_mean.rename(columns = {'success': 'success_mean'}, inplace = True)
success_mean.head(2)

,success_mean
user_id,
0.0,0.0
71.0,1.0


количество выполненных аттестационных активностей

In [1171]:
attestation= df_merged[(df_merged.is_attestation == 1)&(df_merged.obyaz_priznak == 1)&(df_merged.success == 1)]
sum_attestation_id = attestation.groupby(['user_id'])[['success']].agg('count')
sum_attestation_id.rename(columns = {'success': 'attestation_task'}, inplace = True)
sum_attestation_id.head(2)

,attestation_task
user_id,
171.0,6
192.0,6


количество выполненных обязательных активностей

In [1172]:
shed_res_act_1 = df_merged[(df_merged.success == 1)&(df_merged.obyaz_priznak == 1)&(df_merged.is_attestation == 0)]
sum_result_1 = shed_res_act_1.groupby(['user_id'])[['success']].agg('count')
sum_result_1.head(2)

,success
user_id,
71.0,2
171.0,34


количество выполненных необязательных активностей

In [1173]:
shed_res_act_0 = df_merged[(df_merged.success == 1)&(df_merged.obyaz_priznak == 0)&(df_merged.is_attestation == 0)]
sum_result_0 = shed_res_act_0.groupby(['user_id'])[['success']].agg('count')
sum_result_0.head(2)

,success
user_id,
193.0,1
1315.0,23


создаем сводный обзорный датасет типа "один студент - одна строка"

In [1174]:
# прибавляем количество выполненных обязательных и необязательных активностей
df_overwiew = pd.merge(for_overwiew, sum_result_1, on='user_id', how='left')
df_overwiew = pd.merge(df_overwiew, sum_result_0, on='user_id', how='left')
df_overwiew.head(2)

,user_id,course_id,course_progress,course_attestation,course_attestation_date,first_authirization,num_of_auth,webinar_int,w_view_hours,view_task_id,view_activity_id,success_x,success_y
0,71,82,13,Не сдана,NaT,2024-05-27 14:03:55,33.0,0.0,6.602500,119.0,90.0,2.0,NaN
1,171,82,88,100,2024-08-28 13:33:00,2024-06-05 15:21:40,121.0,1.0,14.450556,251.0,496.0,34.0,NaN


In [1175]:
#переименовываем колонки в "обязательные" и "необязательные активности"
df_overwiew.rename(columns = {'success_x': 'required_task', 'success_y': 'optional_task'}, inplace = True)

In [1176]:
# прибавляем количество выполненных аттестационных активностей
df_overwiew = pd.merge(df_overwiew, sum_attestation_id, on = 'user_id', how = 'left')

In [1177]:
# прибавляем средний результат выполнения неаттестационных активностей
df_overwiew = pd.merge(df_overwiew , result_mean, on = 'user_id', how = 'left')

In [1178]:
# прибавляем средний успех выполнения неаттестационных активностей
df_overwiew = pd.merge(df_overwiew, success_mean, on = 'user_id', how = 'left')

In [1179]:
# прибавляем среднюю длительность выполнения неаттестационных активностей
df_overwiew = pd.merge(df_overwiew, delta_days_mean, on = 'user_id', how = 'left')

In [1180]:
# заполняем пропуски
df_overwiew = df_overwiew.fillna(0)
df_overwiew.head(2)

,user_id,course_id,course_progress,course_attestation,course_attestation_date,first_authirization,num_of_auth,webinar_int,w_view_hours,view_task_id,view_activity_id,required_task,optional_task,attestation_task,result_mean,success_mean,delay_days_mean
0,71,82,13,Не сдана,0,2024-05-27 14:03:55,33.0,0.0,6.602500,119.0,90.0,2.0,0.0,0.0,100.00000,1.000000,0.000000
1,171,82,88,100,2024-08-28 13:33:00,2024-06-05 15:21:40,121.0,1.0,14.450556,251.0,496.0,34.0,0.0,6.0,85.74359,0.871795,4.710526


In [1181]:
# создаем обзорный датасет для любого курса (в примере курс "Аналитик данных")
df_overwiew_83 = df_overwiew.query('course_id == 83')

In [1182]:
# рассчитываем прогресс по курсу на основе данных о количестве выполненных обязательных и необязательных активностей
# в знаменателе 24 это количество обязательных активностей по расписанию для курса "Аналитик данных" из строки 1088.
df_overwiew_83.eval('progress_calculated = (required_task + optional_task) / (24 + optional_task )*100', inplace=True)
df_overwiew_83.head(5)

C:\Users\Инга\AppData\Local\Temp\ipykernel_2908\368365231.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_overwiew_83.eval('progress_calculated = (required_task + optional_task) / (24 + optional_task )*100', inplace=True)


,user_id,course_id,course_progress,course_attestation,course_attestation_date,first_authirization,num_of_auth,webinar_int,w_view_hours,view_task_id,view_activity_id,required_task,optional_task,attestation_task,result_mean,success_mean,delay_days_mean,progress_calculated
7,1931,83,100,100,2024-08-25 20:10:00,2024-05-28 20:09:00,177.0,1.0,26.239722,448.0,829.0,24.0,5.0,3.0,98.620690,1.000000,3.275862,100.000000
11,2259,83,100,100,2024-08-30 18:19:00,2024-03-30 13:11:18,136.0,0.0,31.352778,770.0,1252.0,24.0,4.0,3.0,89.677419,0.903226,12.741935,100.000000
19,2872,83,83,100,2024-09-14 11:11:00,2024-05-27 22:21:22,249.0,1.0,29.523056,688.0,1595.0,20.0,3.0,2.0,92.000000,0.920000,11.400000,85.185185
20,2920,83,88,100,2024-08-26 17:25:00,2024-03-30 17:19:53,44.0,0.0,0.006667,203.0,312.0,21.0,3.0,2.0,88.888889,0.888889,11.444444,88.888889
21,2921,83,75,100,2024-08-30 10:25:00,2024-03-30 17:33:45,55.0,0.0,2.380278,298.0,578.0,18.0,4.0,2.0,78.571429,0.785714,11.178571,78.571429


In [1183]:
# проверяем, сколько студентов на курсе 83
df_overwiew_83['user_id'].value_counts()

user_id
1931    1
5510    1
5516    1
5517    1
5518    1
       ..
4386    1
4418    1
4442    1
4451    1
6443    1
Name: count, Length: 182, dtype: int64

Нами получен сводный датасет для 182 студентов курс 83 "Аналитик данных"

In [1184]:
# пример вывода данных по одному студенту курса "Аналитик"
df_overwiew_83.query('user_id == 5596')

,user_id,course_id,course_progress,course_attestation,course_attestation_date,first_authirization,num_of_auth,webinar_int,w_view_hours,view_task_id,view_activity_id,required_task,optional_task,attestation_task,result_mean,success_mean,delay_days_mean,progress_calculated
547,5596,83,100,100,2024-09-01 15:37:00,2024-05-31 22:43:19,204.0,0.0,68.911389,344.0,742.0,24.0,4.0,3.0,96.551724,0.965517,7.137931,100.0


In [1185]:
#Значения колонок
#user_id - идентификационный номер студента
#course_id - идентификационный номер курса
#course_progress	- прогресс по курсу, %
#course_attestation - аттестация (значение в % / не сдана)
#course_attestation_date - дата аттестации	
#first_authirization - дата первой авторизации
#num_of_auth	- общее количество авторизаций
#webinar_int - просмотр водного вебинара (да/нет)
#w_view_hours - общее количество часов просмотра вебинаров
#view_task_id - общее количество просмотренных заданий
#view_activity_id - общее количество просмотренных активностей
#required_task - общее количество выполненных обязательных неаттестационных активностей
#optional_task - общее количество выполненных необязательных неаттестационных активностей
#attestation_task - общее количество выполненных аттестационных активностей	
#result_mean	- средний результат выполнения обязательных неаттестационных активностей, %
#success_mean - средний успех выполнения обязательных неаттестационных активностей (отношение удачных попыток выполнения ко всем попыткам) от 0 до 1
#delay_days_mean	- среднее количество дней, потребовавшихся на выполнение неаттестационных активностей
#progress_calculated - рассчетный прогресс по курсу, %

In [1186]:
# экспортируем сводный датасет для курса 83
#df_overwiew_83.to_csv('E:/Учёба/DA/EduTech/overwiew_83.csv')